In [1]:
#Imports
import gym
import tensorflow as tf
import numpy as np
import random

load_model           = True
environment_name     = "BipedalWalker-v2"
saver_file           = "./models/bipedal"
learning_rate        =.000001
frame_limit          = 400
discount_decay_rate  = .95
n_epochs             = 5001
random_action_probability_range  = [.05, .20]
random_action_repeat_steps_range = [ 5,  20]

    
def helper_discount_rewards(rewards, discount_rate):
    #Takes in rewards and applies discount rate
    discounted_rewards = np.zeros(len(rewards))
    cumulative_rewards = 0
    for step in reversed(range(len(rewards))):
        cumulative_rewards = rewards[step] + cumulative_rewards * discount_rate
        discounted_rewards[step] = cumulative_rewards

    relevant_discounted_rewards = discounted_rewards

    reward_mean = relevant_discounted_rewards.mean()
    reward_std = relevant_discounted_rewards.std()

    return [(discounted_reward - reward_mean)/reward_std for discounted_reward in discounted_rewards]

# def helper_discount_rewards(rewards, discount_rate, begin_index, end_index):
#     #Takes in rewards and applies discount rate
#     discounted_rewards = []
    
#     for i in range(len(rewards)):
#         this_reward = 0.0
#         exponent = 1
#         for j in range(i,len(rewards)):
#             this_reward += rewards[j]*discount_rate**exponent
#             exponent += 1
#         discounted_rewards.append(this_reward)
#     return discounted_rewards
    
reward_test = [1,1,1,0,1,1,1]
print(helper_discount_rewards(reward_test, 0.9))



[1.5444718295843003, 0.9837496933802429, 0.36072509759795657, -0.3315244532712502, -0.08340961623176471, -0.8250074686376072, -1.6490050824218765]


In [2]:
# build the NN
reg_param = 0.00001

value_scale = 0.5  #how much does the value vs progressive gains influence the loss

activ = tf.nn.elu

n_inputs = 4
n_outputs = 4
n_obs_inputs = 24

n_hidden1 = 256
n_hidden2 = 256
n_hidden3 = 256
n_hidden4 = 256
n_hidden5 = 256
n_hidden6 = 128
n_hidden7 = 64
n_hidden8 = 32
initializer = tf.contrib.layers.variance_scaling_initializer()

tf.reset_default_graph()

tf_input_obs = tf.placeholder(tf.float32, shape=(None, n_obs_inputs))
tf_input_action = tf.placeholder(tf.float32, shape=(None, n_inputs))
tf_input_learning_rate = tf.placeholder(tf.float32)
tf_input_reward = tf.placeholder(tf.float32)

input_layer = tf.layers.dense(tf_input_obs, n_obs_inputs, activation=activ, name="input", kernel_initializer=initializer)

hidden1 = tf.layers.dense(input_layer, n_hidden1, activation=activ, name="hidden1", kernel_initializer=initializer)
hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=activ, name="hidden2", kernel_initializer=initializer)
hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=activ, name="hidden3", kernel_initializer=initializer)
hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=activ, name="hidden4", kernel_initializer=initializer)
hidden5 = tf.layers.dense(hidden4, n_hidden5, activation=activ, name="hidden5", kernel_initializer=initializer)
hidden6 = tf.layers.dense(hidden5, n_hidden6, activation=activ, name="hidden6", kernel_initializer=initializer)
hidden7 = tf.layers.dense(hidden6, n_hidden7, activation=activ, name="hidden7", kernel_initializer=initializer)
hidden8 = tf.layers.dense(hidden7, n_hidden8, activation=activ, name="hidden8", kernel_initializer=initializer)

logits = tf.layers.dense(hidden8, n_outputs, name="output")
#value  = tf.layers.dense(hidden8, 1,         name='value')

#pg_loss    = tf.reduce_mean(1.0 - tf_input_reward) * tf.reduce_mean(tf.square(logits-tf_input_action))
#value_loss = value_scale * tf.reduce_mean(tf.square(1.0- tf_input_reward))
#loss = pg_loss + value_loss

input_loss = tf.reduce_mean(tf.square(logits-tf_input_action))
#loss = tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=tf_input_action)
#value_loss = 0.5 * tf.reduce_sum(tf.square(tf_input_reward - tf.reshape(value,[-1])))
reward_loss = 0.5 * tf_input_reward
loss = tf.reduce_sum(reward_loss + input_loss)
optimizer = tf.train.AdamOptimizer(tf_input_learning_rate)
training_op = optimizer.minimize(loss)

# regularization loss
#policy_network_variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="policy_network")
        
# compute policy loss and regularization loss
# cross_entropy_loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_input_action)
# pg_loss            = tf.reduce_mean(cross_entropy_loss)
# reg_loss           = tf.reduce_sum([tf.reduce_sum(tf.square(x)) for x in policy_network_variables])
# loss               = pg_loss + reg_param * reg_loss

# gradients = optimizer.compute_gradients(loss)

# compute policy gradients
# for i, (grad, var) in enumerate(gradients):
#     if grad is not None:
#         gradients[i] = (grad * tf_input_reward, var)


# training_op = optimizer.apply_gradients(gradients)

init = tf.global_variables_initializer()
saver = tf.train.Saver()


In [3]:
env = gym.make(environment_name)
global_step = 0

with tf.Session() as sess:
    if load_model:
        print("Loading existing model before training: {}".format(saver_file))
        saver.restore(sess, saver_file)
    else:
        print("Creating new model before training: {}".format(saver_file))
        sess.run(init)
    
    for epoch in range(n_epochs):  #play n_epochs games
        game_step_counter = 0
        random_action_counter = 0
        
        all_actions = []
        all_rewards = []
        all_observations = []
        all_logits = []

        observation = env.reset()
        input_action = env.action_space.sample()  #set an initial input value
        
        while True:
            random_action_probability  = random.uniform(random_action_probability_range[0], random_action_probability_range[1])
            random_action_repeat_steps = random.randint(random_action_repeat_steps_range[0], random_action_repeat_steps_range[1])

            if epoch % 10 == 0:
                random_action_probability = 0.0
                
            #feed data into the AI to get action from the AI
            feed_dict = {tf_input_obs    : np.reshape(observation, (1, len(observation))), 
                         tf_input_action : np.reshape(input_action, (1, len(input_action))),
                         tf_input_learning_rate: 0.0,
                         tf_input_reward: 0.0}
            logits_out = sess.run([logits], feed_dict=feed_dict)

            # set the input value to feed into the next step
            
            if random_action_counter > 0 and random_action_counter < random_action_repeat_steps:
                random_action_counter += 1
            elif random_action_counter >= random_action_repeat_steps:
                random_action_counter = 0
            elif random.uniform(0.0, 1.0) < random_action_probability:
                input_action = env.action_space.sample()
                random_action_counter += 1
                #print("Random action {}: ".format(input_action))
            else:
                input_action = logits_out[0][0]

            #run the next step given the input from the logits
            observation, reward_float, done_bool, info_dict = env.step(input_action)
                
            #add the data to our lists
            all_observations.append(observation)
            all_logits.append(logits_out[0][0])
            all_actions.append(input_action)
            all_rewards.append(reward_float)

            if done_bool:
                break
                
            if game_step_counter > frame_limit:
                break

            if epoch % 1 == 0:
                env.render()  #display the current frame.
               
            game_step_counter += 1
        
        if epoch % 50 == 0 and epoch > 0:
            print("Saving model at epoch {}: {}".format(epoch, saver_file))
            saver.save(sess, saver_file)

        num_frames = len(all_actions)
        discounted_rewards = helper_discount_rewards(all_rewards, discount_decay_rate) #-1-frames_to_skip_end
        discounted_rewards_median = np.median(discounted_rewards)
        discounted_rewards_mean = np.mean(discounted_rewards)
        
        discounted_rewards_max = np.amax(discounted_rewards)
        
        loss_out_sum = 0.0
        all_losses = []
        all_loss_adders = []
        training_frame_counter = 0
        #use stored data to train
        for i in range(num_frames): 
            train_input_action = all_actions[i]
            
#             if discounted_rewards[i] + all_rewards[i] >= 1.0:
#                 train_input_action = all_actions[i]
#                 training_frame_counter += 1
#             else:
#                 continue
            
            loss_adder = discounted_rewards[i] - discounted_rewards_max
            if discounted_rewards[i] + all_rewards[i] >= 0:
                loss_adder = 0
    
            feed_dict = {tf_input_obs : np.reshape(all_observations[i], (1, len(all_observations[i]))), 
                         tf_input_action : np.reshape(train_input_action, (1, len(train_input_action))),
                         tf_input_learning_rate: learning_rate,
                         tf_input_reward: loss_adder}
        
            loss_out, _, logits_out = sess.run([loss, training_op, logits], feed_dict=feed_dict)
            loss_out_sum += abs(loss_out)
            all_losses.append(loss_out)
            all_loss_adders.append(loss_adder)

        print("Epoch: " + str(epoch) + ", rwd total: " + str(round(np.sum(all_rewards), 2)) + ", avg loss: " + str(round(loss_out_sum/num_frames, 2)) + ", rand prob: " + str(random_action_probability))

        #TODO - should learning rate decrease over time?
        print("")


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Loading existing model before training: ./models/bipedal
INFO:tensorflow:Restoring parameters from ./models/bipedal
Epoch: 0, rwd total: -107.2, avg loss: 0.25, rand prob: 0.0

Epoch: 1, rwd total: -115.71, avg loss: 0.35, rand prob: 0.1341494859239328

Epoch: 2, rwd total: -109.34, avg loss: 0.38, rand prob: 0.11466175438453255

Epoch: 3, rwd total: -107.24, avg loss: 0.47, rand prob: 0.08426870632714331

Epoch: 4, rwd total: -110.18, avg loss: 0.46, rand prob: 0.13657842438289816

Epoch: 5, rwd total: -106.33, avg loss: 0.42, rand prob: 0.0732504196770238

Epoch: 6, rwd total: -113.26, avg loss: 0.44, rand prob: 0.05678063517222935

Epoch: 7, rwd total: -136.47, avg loss: 0.31, rand prob: 0.0920195054801734

Epoch: 8, rwd total: -95.16, avg loss: 0.32, rand prob: 0.051119533488494254

Epoc

Epoch: 102, rwd total: -103.18, avg loss: 0.44, rand prob: 0.12495029490380555

Epoch: 103, rwd total: -135.11, avg loss: 0.32, rand prob: 0.07633786743923394

Epoch: 104, rwd total: -105.1, avg loss: 0.32, rand prob: 0.10574704179027099

Epoch: 105, rwd total: -111.5, avg loss: 0.44, rand prob: 0.05278783856016184

Epoch: 106, rwd total: -106.37, avg loss: 0.34, rand prob: 0.11360058698692041

Epoch: 107, rwd total: -117.74, avg loss: 0.47, rand prob: 0.11289331324569749

Epoch: 108, rwd total: -113.18, avg loss: 0.52, rand prob: 0.06214242240791516

Epoch: 109, rwd total: -112.97, avg loss: 0.44, rand prob: 0.14861370159176987

Epoch: 110, rwd total: -107.75, avg loss: 0.24, rand prob: 0.0

Epoch: 111, rwd total: -120.89, avg loss: 0.29, rand prob: 0.11690563512904122

Epoch: 112, rwd total: -115.36, avg loss: 0.4, rand prob: 0.0977187545185573

Epoch: 113, rwd total: -118.87, avg loss: 0.28, rand prob: 0.19873877270830198

Epoch: 114, rwd total: -118.16, avg loss: 0.47, rand prob: 0

Epoch: 206, rwd total: -106.06, avg loss: 0.38, rand prob: 0.1993479953902973

Epoch: 207, rwd total: -123.72, avg loss: 0.35, rand prob: 0.07560063972233788

Epoch: 208, rwd total: -113.19, avg loss: 0.49, rand prob: 0.10365652303674819

Epoch: 209, rwd total: -119.71, avg loss: 0.4, rand prob: 0.1974094391920026

Epoch: 210, rwd total: -128.57, avg loss: 0.24, rand prob: 0.0

Epoch: 211, rwd total: -125.37, avg loss: 0.26, rand prob: 0.17996905452504375

Epoch: 212, rwd total: -121.64, avg loss: 0.39, rand prob: 0.16487988493724087

Epoch: 213, rwd total: -100.83, avg loss: 0.33, rand prob: 0.1970467176594876

Epoch: 214, rwd total: -120.71, avg loss: 0.3, rand prob: 0.13075109067572976

Epoch: 215, rwd total: -109.53, avg loss: 0.34, rand prob: 0.12302832639963047

Epoch: 216, rwd total: -106.21, avg loss: 0.47, rand prob: 0.165744235255711

Epoch: 217, rwd total: -101.22, avg loss: 0.32, rand prob: 0.19193363737142943

Epoch: 218, rwd total: -102.81, avg loss: 0.29, rand prob: 0.07

Epoch: 310, rwd total: -130.14, avg loss: 0.23, rand prob: 0.0

Epoch: 311, rwd total: -106.44, avg loss: 0.32, rand prob: 0.1551153719391609

Epoch: 312, rwd total: -122.34, avg loss: 0.38, rand prob: 0.11039976872986616

Epoch: 313, rwd total: -102.69, avg loss: 0.26, rand prob: 0.11504174836708032

Epoch: 314, rwd total: -128.03, avg loss: 0.32, rand prob: 0.19476961465321274

Epoch: 315, rwd total: -101.63, avg loss: 0.3, rand prob: 0.11191237846395648

Epoch: 316, rwd total: -106.44, avg loss: 0.39, rand prob: 0.10621053930330894

Epoch: 317, rwd total: -111.55, avg loss: 0.35, rand prob: 0.12097637032658748

Epoch: 318, rwd total: -121.95, avg loss: 0.43, rand prob: 0.08663240545866681

Epoch: 319, rwd total: -114.96, avg loss: 0.41, rand prob: 0.17072165277580323

Epoch: 320, rwd total: -127.53, avg loss: 0.27, rand prob: 0.0

Epoch: 321, rwd total: -123.83, avg loss: 0.31, rand prob: 0.11444007709082923

Epoch: 322, rwd total: -106.95, avg loss: 0.46, rand prob: 0.0685532102339

Epoch: 415, rwd total: -122.71, avg loss: 0.31, rand prob: 0.12536767130625862

Epoch: 416, rwd total: -113.52, avg loss: 0.35, rand prob: 0.06911467034175002

Epoch: 417, rwd total: -118.71, avg loss: 0.42, rand prob: 0.12339894294097281

Epoch: 418, rwd total: -116.18, avg loss: 0.33, rand prob: 0.1758188991075823

Epoch: 419, rwd total: -12.7, avg loss: 1.1, rand prob: 0.13264203933028268

Epoch: 420, rwd total: -105.8, avg loss: 0.36, rand prob: 0.0

Epoch: 421, rwd total: -103.43, avg loss: 0.32, rand prob: 0.05051090199037189

Epoch: 422, rwd total: -98.67, avg loss: 0.41, rand prob: 0.12583595713747042

Epoch: 423, rwd total: -108.75, avg loss: 0.4, rand prob: 0.097416195259944

Epoch: 424, rwd total: -103.69, avg loss: 0.25, rand prob: 0.10830083149137353

Epoch: 425, rwd total: -109.76, avg loss: 0.42, rand prob: 0.08694867776452164

Epoch: 426, rwd total: -99.51, avg loss: 0.29, rand prob: 0.07575506563214097

Epoch: 427, rwd total: -118.13, avg loss: 0.45, rand prob: 0.14659

...

Epoch: 4545, rwd total: -110.93, avg loss: 0.39, rand prob: 0.064433004991388

Epoch: 4546, rwd total: -111.71, avg loss: 0.31, rand prob: 0.17390158733151656

Epoch: 4547, rwd total: -99.28, avg loss: 0.42, rand prob: 0.11378830689351499

Epoch: 4548, rwd total: -101.37, avg loss: 0.38, rand prob: 0.15536044372356267

Epoch: 4549, rwd total: -118.94, avg loss: 0.28, rand prob: 0.08910820759122012

Saving model at epoch 4550: ./models/bipedal
Epoch: 4550, rwd total: -100.03, avg loss: 0.38, rand prob: 0.0

Epoch: 4551, rwd total: -97.69, avg loss: 0.35, rand prob: 0.1821895811030006

Epoch: 4552, rwd total: -117.08, avg loss: 0.41, rand prob: 0.05358756134735226

Epoch: 4553, rwd total: -10.99, avg loss: 1.38, rand prob: 0.08275210541480763

Epoch: 4554, rwd total: -113.89, avg loss: 0.49, rand prob: 0.16529423828491244

Epoch: 4555, rwd total: -122.07, avg loss: 0.35, rand prob: 0.15517970410071602

Epoch: 4556, rwd total: -119.89, avg loss: 0.53, rand prob: 0.06660440103160506

Epoch

Epoch: 4648, rwd total: -100.08, avg loss: 0.41, rand prob: 0.13646041325778524

Epoch: 4649, rwd total: -115.71, avg loss: 0.49, rand prob: 0.1377421245888547

Saving model at epoch 4650: ./models/bipedal
Epoch: 4650, rwd total: -101.03, avg loss: 0.4, rand prob: 0.0

Epoch: 4651, rwd total: -112.93, avg loss: 0.53, rand prob: 0.07415393389219488

Epoch: 4652, rwd total: -106.32, avg loss: 0.46, rand prob: 0.1712456628533167

Epoch: 4653, rwd total: -98.15, avg loss: 0.35, rand prob: 0.15086187083996722

Epoch: 4654, rwd total: -107.16, avg loss: 0.39, rand prob: 0.1584423162085075

Epoch: 4655, rwd total: -99.32, avg loss: 0.32, rand prob: 0.16478914847208032

Epoch: 4656, rwd total: -106.01, avg loss: 0.4, rand prob: 0.10799691067782864

Epoch: 4657, rwd total: -114.64, avg loss: 0.43, rand prob: 0.09589609428117539

Epoch: 4658, rwd total: -104.94, avg loss: 0.37, rand prob: 0.14714716444924625

Epoch: 4659, rwd total: -102.46, avg loss: 0.43, rand prob: 0.08491212053398572

Epoch:

Epoch: 4750, rwd total: -100.82, avg loss: 0.4, rand prob: 0.0

Epoch: 4751, rwd total: -98.99, avg loss: 0.36, rand prob: 0.07654718355013225

Epoch: 4752, rwd total: -109.21, avg loss: 0.44, rand prob: 0.1557409189783503

Epoch: 4753, rwd total: -107.99, avg loss: 0.34, rand prob: 0.09754147844811242

Epoch: 4754, rwd total: -112.3, avg loss: 0.44, rand prob: 0.09669737542342675

Epoch: 4755, rwd total: -124.84, avg loss: 0.27, rand prob: 0.13148873409915052

Epoch: 4756, rwd total: -110.14, avg loss: 0.5, rand prob: 0.09600568127924083

Epoch: 4757, rwd total: -98.79, avg loss: 0.35, rand prob: 0.06069751976417302

Epoch: 4758, rwd total: -119.15, avg loss: 0.48, rand prob: 0.19803249638364107

Epoch: 4759, rwd total: -119.0, avg loss: 0.37, rand prob: 0.1617436727733088

Epoch: 4760, rwd total: -130.55, avg loss: 0.24, rand prob: 0.0

Epoch: 4761, rwd total: -112.85, avg loss: 0.32, rand prob: 0.07401033675258159

Epoch: 4762, rwd total: -127.29, avg loss: 0.4, rand prob: 0.0583711

Epoch: 4853, rwd total: -103.94, avg loss: 0.4, rand prob: 0.1704808831726362

Epoch: 4854, rwd total: -109.5, avg loss: 0.48, rand prob: 0.16551501348307962

Epoch: 4855, rwd total: -30.96, avg loss: 0.8, rand prob: 0.19628779114466233

Epoch: 4856, rwd total: -104.53, avg loss: 0.37, rand prob: 0.14857125293117723

Epoch: 4857, rwd total: -109.4, avg loss: 0.44, rand prob: 0.08287760581305825

Epoch: 4858, rwd total: -113.03, avg loss: 0.57, rand prob: 0.19832056827581634

Epoch: 4859, rwd total: -100.54, avg loss: 0.38, rand prob: 0.12260581926548529

Epoch: 4860, rwd total: -100.86, avg loss: 0.39, rand prob: 0.0

Epoch: 4861, rwd total: -97.58, avg loss: 0.36, rand prob: 0.11697234938358102

Epoch: 4862, rwd total: -106.04, avg loss: 0.38, rand prob: 0.10108536929266523

Epoch: 4863, rwd total: -125.1, avg loss: 0.31, rand prob: 0.0725915986687461

Epoch: 4864, rwd total: -127.14, avg loss: 0.3, rand prob: 0.06268677876470709

Epoch: 4865, rwd total: -19.78, avg loss: 0.84, rand p

Epoch: 4956, rwd total: -103.34, avg loss: 0.36, rand prob: 0.12403771497085608

Epoch: 4957, rwd total: -20.52, avg loss: 0.73, rand prob: 0.11008623006646667

Epoch: 4958, rwd total: -106.36, avg loss: 0.47, rand prob: 0.192694966871309

Epoch: 4959, rwd total: -107.5, avg loss: 0.41, rand prob: 0.0720005609504949

Epoch: 4960, rwd total: -99.44, avg loss: 0.3, rand prob: 0.0

Epoch: 4961, rwd total: -110.34, avg loss: 0.46, rand prob: 0.06869313162532044

Epoch: 4962, rwd total: -115.65, avg loss: 0.37, rand prob: 0.17548348137316833

Epoch: 4963, rwd total: -99.84, avg loss: 0.29, rand prob: 0.1798647440941536

Epoch: 4964, rwd total: -114.98, avg loss: 0.35, rand prob: 0.13953572260787522

Epoch: 4965, rwd total: -124.82, avg loss: 0.29, rand prob: 0.12881580179530894

Epoch: 4966, rwd total: -130.72, avg loss: 0.29, rand prob: 0.16691053476540751

Epoch: 4967, rwd total: -106.65, avg loss: 0.48, rand prob: 0.06038314915892683

Epoch: 4968, rwd total: -111.54, avg loss: 0.38, rand

In [7]:
print(num_frames)
print(len(all_observations))
print(len(all_actions))
print(len(all_rewards))
print(len(all_logits))
print(len(discounted_rewards))
print(len(all_loss_adders))


values = discounted_rewards

for this_value1, this_value2, this_value_3 in zip(all_logits, all_loss_adders, all_rewards):
    print("{}, {}, {}".format(this_value1, this_value2, this_value_3))


68
68
68
68
68
68
68
[ 0.33399928  0.49272266 -0.23483634  0.44599265], 0, -0.05175616260617972
[ 0.32071358  0.4921382  -0.21540529  0.4469517 ], 0, -0.04350652573506156
[ 0.37528372  0.49938813 -0.35571864  0.44664004], 0, -0.037606954952079864
[ 0.3450482   0.5041796  -0.38113162  0.45256233], 0, -0.044626839242875575
[ 0.30360532  0.50542474 -0.43276712  0.4560808 ], 0, -0.05070750096440315
[ 0.23425584  0.521073   -0.3965598   0.4615007 ], 0, -0.05592019775261718
[ 0.15533109  0.5333569  -0.38124308  0.47286797], 0, -0.07413928828636568
[ 0.03977159  0.5487703  -0.35491726  0.48616666], 0, -0.0896236710300072
[ 0.02831535  0.5501761  -0.36589625  0.48787957], 0, -0.08857485595345498
[ 0.04353684  0.54838294 -0.36761585  0.4861381 ], 0, -0.08969329470396042
[ 0.08899294  0.53946984 -0.37121043  0.4829104 ], 0, -0.08659402571121616
[ 0.09789293  0.5390068  -0.37961385  0.48225576], 0, -0.08282382069031517
[ 0.10794453  0.5381584  -0.3880082   0.48146835], 0, -0.0786551425655671
[ 0.

In [3]:

env = gym.make(environment_name)
observation = env.reset()

print("obs.shape: {}".format(observation.shape)) #obs.shape: (210, 160, 3)
print("env.action_space: {}".format(env.action_space)) #env.action_space: Discrete(9)


for _ in range(14):
    env.render()
    action = env.action_space.sample()
    print("action: {}".format(action))
    print(type(action))
    observation, reward_float, done_bool, info_dict = env.step(action) # take a random action
    
    print("observation: {}".format(observation))
    print("reward_float: {}".format(reward_float))
    print("done_bool: {}".format(done_bool))
    print("info_dict: {}".format(info_dict))

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
obs.shape: (24,)
env.action_space: Box(4,)
action: [0.09762701 0.43037874 0.20552675 0.08976637]
<class 'numpy.ndarray'>
observation: [ 0.00245932 -0.00691658  0.00590502  0.01961307 -0.29061297 -0.71013689
  1.4722293   0.99358988  1.          0.30079773 -0.01692069  0.1633203
  0.33325795  1.          0.45289648  0.45803979  0.4740701   0.50296849
  0.54874223  0.61897415  0.72858626  0.91021472  1.          1.        ]
reward_float: -0.01773284123155096
done_bool: False
info_dict: {}
action: [-0.1526904   0.29178822 -0.12482557  0.78354603]
<class 'numpy.ndarray'>
observation: [ 0.00255965  0.00256462  0.00806203  0.00679228 -0.01382717 -0.39574927
  1.08375649  0.          1.          0.26067981 -0.41715193  0.26995051
  0.24234966  1.          0.45600694  0.46118557  0.47732601  0.50642